In [94]:
import dask.dataframe as dd
import pandas as pd
from collections import OrderedDict

In [3]:
df = dd.read_csv("data/titanic.csv")

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


These functions come from the data_summary() class in my DataExplorationTool project. They didn't work natively switching from pandas to dask. The versions below work with the dask representation of the csv.

In [38]:
def get_num_records(data):
    return len(data)

In [44]:
num_records = get_num_records(df)
num_records

891

In [40]:
def get_num_features(data):
    return len(list(df.columns))

In [45]:
num_features = get_num_features(df)
num_features

12

In [96]:
def count_missing(data, num_records):
    
    # Count the number of columns missing for each row
    missing = data.apply(lambda x: sum(x.isnull().values), axis=1)
    data= data.assign(num_missing = missing)
    
    num_rows_missing = {}
    cumulative_row_sum = 0

    for i in range(0, num_records):
        # Count the number of missing rows and add it to the list
        num_rows_i_missing = len(data[data.num_missing == i])
        num_rows_missing[i] = num_rows_i_missing
        cumulative_row_sum += num_rows_i_missing

        # If we reached the point where we have accounted for all rows, stop looking for rows
        if cumulative_row_sum == num_records:
            break

    # Sort the dictionary
    num_rows_missing = OrderedDict(sorted(num_rows_missing.items()))

    # Remove the added column
    data = data.drop("num_missing", axis=1)

    return num_rows_missing

In [83]:
missing = df.apply(lambda x: sum(x.isnull().values), axis=1)

/Users/margaretpearce/.virtualenvs/dask/lib/python3.5/site-packages/dask/dataframe/core.py:2553: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [85]:
data = df.assign(num_missing = missing)

In [86]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,num_missing
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [88]:
len(data[data.num_missing == 0])

183

In [89]:
len(data[data.num_missing == 1])

550

In [90]:
len(data[data.num_missing == 2])

158

In [91]:
183+550+158

891

In [97]:
count_missing(df, num_records)

/Users/margaretpearce/.virtualenvs/dask/lib/python3.5/site-packages/dask/dataframe/core.py:2553: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


OrderedDict([(0, 183), (1, 550), (2, 158)])

In [71]:
def get_sample(data, num_records, features_list):
    num_samples = 5
    if num_records < num_samples:
        num_samples = num_records
        
    # Dask takes a percentage instead of a fixed number
    percent_sample = num_samples / float(num_records)

    # Get the sample using Dask df
    sample_df = data.sample(percent_sample)
    
    # Find out the number of records in the sample
    actual_num_samples = len(sample_df)
    
    # Get the sample as a list (should be 5 records or less)
    sample_list = sample_df.head(actual_num_samples)[features_list].values.tolist()
    
    return sample_list

In [72]:
columns_list = list(df.columns)
columns_list

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [73]:
sample = get_sample(df, num_records, columns_list)
sample

[[567,
  0,
  3,
  'Stoytcheff, Mr. Ilia',
  'male',
  19.0,
  0,
  0,
  '349205',
  7.8958,
  nan,
  'S'],
 [81,
  0,
  3,
  'Waelens, Mr. Achille',
  'male',
  22.0,
  0,
  0,
  '345767',
  9.0,
  nan,
  'S'],
 [384,
  1,
  1,
  'Holverson, Mrs. Alexander Oskar (Mary Aline Towner)',
  'female',
  35.0,
  1,
  0,
  '113789',
  52.0,
  nan,
  'S'],
 [218,
  0,
  2,
  'Jacobsohn, Mr. Sidney Samuel',
  'male',
  42.0,
  1,
  0,
  '243847',
  27.0,
  nan,
  'S'],
 [563,
  0,
  2,
  'Norman, Mr. Robert Douglas',
  'male',
  28.0,
  0,
  0,
  '218629',
  13.5,
  nan,
  'S']]